## Crossover

### Simulated Binary Crossover ('real_sbx')

Details about the crossover can be found in <cite data-cite="sbx"></cite>. Real values can be representated in a binary notation and then a the point crossovers can be performed. SBX simulated this operation by using a probability distribution *simulating* the binary crossover.

### Point Crossover ('real_point', 'bin_point', 'int_point' )


Text...

### Uniform Crossover ('real_ux', 'bin_ux', 'int_ux')


Text...

### Half Uniform Crossover ('bin_hux', 'int_hux')


Text...

### Exponential Crossover ('real_exp', 'bin_exp', 'int_exp')


Text...

### Differential Crossover ('real_de')


Text...

### API